In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import holidays
import datetime

In [ ]:
df: pd.DataFrame = pd.read_excel('data/01 Call-Center-Dataset.xlsx')

df.head()

In [ ]:
dayly: pd.DataFrame = df[['Call Id', 'Date']].groupby('Date').count()
dayly.rename(columns={'Call Id': 'Calls'}, inplace=True)
dayly.index = pd.to_datetime(dayly.index)

dayly.head()

In [ ]:
plt.figure(figsize=(16, 3))
plt.plot(dayly.index, dayly['Calls'])
plt.xlabel('Data')
plt.ylabel('Quantidade de Chamadas')
plt.title('Chamadas por Dia')
plt.xticks(dayly.index[::8])
plt.show()

In [ ]:
# import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import kruskal

# 1. Plot por dia da semana
dayly.groupby(dayly.index.dayofweek).mean().plot(kind='bar')

# # 2. ACF
plot_acf(dayly, lags=7)  # Verifique lags 7, 14, 21
plot_acf(dayly, lags=14)  # Verifique lags 7, 14, 21
plot_acf(dayly, lags=21)  # Verifique lags 7, 14, 21

# # 3. Decomposição
result = seasonal_decompose(dayly, model='additive', period=7)
result.plot()

# 4. Teste de Kruskal-Wallis
groups = [dayly[dayly.index.dayofweek == day] for day in range(7)]
stat, pval = kruskal(*groups)
print(f'p-valor para sazonalidade semanal: {pval}')

In [ ]:
dayly['DayName'] = dayly.index.day_name()

# Calcular as médias de chamadas por dia da semana
weekly_means = dayly.groupby('DayName')['Calls'].mean().round(2)

# Calcular o desvio padrão das médias
std_between_days = weekly_means.std()
mean_calls = weekly_means.mean()
cv = (std_between_days / mean_calls) * 100

print("Médias de chamadas por dia da semana:")
print(weekly_means)
print(f"\nCoeficiente de variação: {cv:.2f}%")

# Realizar teste ANOVA

# Criar listas de chamadas para cada dia da semana
days_data = [dayly[dayly['DayName'] == day]['Calls'] for day in weekly_means.index]
f_stat, p_value = stats.f_oneway(*days_data)

print(f"\nResultados ANOVA:")
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")

stat, pval = stats.kruskal(*days_data)
print(f"p-valor para sazonalidade semanal: {pval:.4f}")

In [ ]:
# Feriados brasileiros do ano de 2021
br_holidays: holidays.BR = holidays.BR(years=2021)

# Convertendo para pandas
holiday_df: pd.DataFrame = pd.DataFrame.from_dict(br_holidays.items())
holiday_df.columns = ['Date', 'Holiday']
holiday_df.set_index('Date', inplace=True)

holiday_df

In [ ]:
# Adiciona Carnaval e Corpus Christi manualmente
holiday_df.loc[datetime.date(2021, 2, 15)] = 'Carnaval'
holiday_df.loc[datetime.date(2021, 2, 16)] = 'Carnaval'
holiday_df.loc[datetime.date(2021, 6, 3)] = 'Corpus Christi'
holiday_df.sort_index(inplace=True)

holiday_df